In [7]:
import tkinter as tk
from tkinter import ttk
from experta import *

class UserProfile(Fact):
    """Information about the user"""
    pass

class Meal(Fact):
    """Meal information including macros, ingredients, and preparation method"""
    pass

class NutritionExpert(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.meal_plan = ""

    @Rule(UserProfile(gender='male', age=MATCH.age, weight=MATCH.weight, height=MATCH.height))
    def calculate_bmr_male(self, age, weight, height):
        bmr = 88.362 + (13.4 * weight) + (4.8 * height) - (5.6 * age)
        self.declare(Fact(bmr=bmr))
        self.bmr = bmr

    @Rule(UserProfile(gender='female', age=MATCH.age, weight=MATCH.weight, height=MATCH.height))
    def calculate_bmr_female(self, age, weight, height):
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.33 * age)
        self.declare(Fact(bmr=bmr))
        self.bmr = bmr

    @Rule(UserProfile(gender='female', age=MATCH.age, weight=MATCH.weight, height=MATCH.height, pregnancy=MATCH.pregnancy))
    def adjust_bmr_pregnancy(self, age, weight, height, pregnancy):
        if pregnancy == 'yes':
            bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.33 * age) + 300  # Additional 300 calories for pregnancy
            self.declare(Fact(bmr=bmr))
            self.bmr = bmr

    @Rule(UserProfile(weight=MATCH.weight, height=MATCH.height))
    def calculate_bmi(self, weight, height):
        bmi = weight / ((height / 100) ** 2)
        self.declare(Fact(bmi=bmi))
        self.bmi = bmi

    @Rule(Fact(bmr=MATCH.bmr), UserProfile(activity_level='none'))
    def calculate_tdee_none(self, bmr):
        tdee = bmr * 1.2
        self.declare(Fact(tdee=tdee))
        self.tdee = tdee

    @Rule(Fact(bmr=MATCH.bmr), UserProfile(activity_level='light'))
    def calculate_tdee_light(self, bmr):
        tdee = bmr * 1.375
        self.declare(Fact(tdee=tdee))
        self.tdee = tdee

    @Rule(Fact(bmr=MATCH.bmr), UserProfile(activity_level='moderate'))
    def calculate_tdee_moderate(self, bmr):
        tdee = bmr * 1.55
        self.declare(Fact(tdee=tdee))
        self.tdee = tdee

    @Rule(Fact(bmr=MATCH.bmr), UserProfile(activity_level='very_active'))
    def calculate_tdee_very_active(self, bmr):
        tdee = bmr * 1.725
        self.declare(Fact(tdee=tdee))
        self.tdee = tdee

    @Rule(Fact(bmr=MATCH.bmr), UserProfile(activity_level='extremely_active'))
    def calculate_tdee_extremely_active(self, bmr):
        tdee = bmr * 1.9
        self.declare(Fact(tdee=tdee))
        self.tdee = tdee

    @Rule(Fact(tdee=MATCH.tdee), UserProfile(goal='maintenance'))
    def calculate_calories_maintenance(self, tdee):
        cal = tdee
        self.declare(Fact(cal=cal))
        self.cal = cal

    @Rule(Fact(tdee=MATCH.tdee), UserProfile(goal='bulking'))
    def calculate_calories_bulking(self, tdee):
        cal = tdee + 500
        self.declare(Fact(cal=cal))
        self.cal = cal

    @Rule(Fact(tdee=MATCH.tdee), UserProfile(goal='cutting'))
    def calculate_calories_cutting(self, tdee):
        cal = tdee - 500
        self.declare(Fact(cal=cal))
        self.cal = cal

    @Rule(Fact(cal=MATCH.cal), UserProfile(goal=MATCH.goal))
    def calculate_macros(self, cal, goal):
        if goal == 'bulking':
            protein = cal * 0.225 / 4
            carbs = cal * 0.5 / 4
            fat = cal * 0.225 / 9
        elif goal == 'cutting':
            protein = cal * 0.335 / 4
            carbs = cal * 0.425 / 4
            fat = cal * 0.225 / 9
        elif goal == 'maintenance':
            protein = cal * 0.275 / 4
            carbs = cal * 0.45 / 4
            fat = cal * 0.275 / 9
        
        self.protein = protein
        self.carbs = carbs
        self.fat = fat

    @Rule(UserProfile())
    def initialize_meals(self):
        meals = [
            Meal(name="Oatmeal with Fruits", protein=10, carbs=55, fat=5, 
                 ingredients="Oats, Banana, Berries", 
                 preparation="Cook oats, add sliced banana and berries", 
                 suitable_for=["Diabetes", "Hypertension"]),
            Meal(name="Grilled Chicken Salad", protein=30, carbs=10, fat=10, 
                 ingredients="Chicken breast, Lettuce, Olive oil", 
                 preparation="Grill chicken, mix with lettuce, drizzle olive oil", 
                 suitable_for=["Diabetes", "Heart Disease"]),
            Meal(name="Quinoa and Veggies", protein=15, carbs=40, fat=5, 
                 ingredients="Quinoa, Mixed Vegetables", 
                 preparation="Cook quinoa, stir-fry vegetables", 
                 suitable_for=["Celiac Disease", "IBS"]),
            Meal(name="Greek Yogurt with Nuts", protein=15, carbs=10, fat=15, 
                 ingredients="Greek yogurt, Mixed nuts", 
                 preparation="Mix yogurt with nuts", 
                 suitable_for=["Hypertension"]),
            Meal(name="Baked Salmon with Asparagus", protein=25, carbs=5, fat=20, 
                 ingredients="Salmon, Asparagus, Olive oil", 
                 preparation="Bake salmon with asparagus and olive oil", 
                 suitable_for=["Heart Disease"]),
            Meal(name="Fruit Smoothie", protein=5, carbs=40, fat=2, 
                 ingredients="Mixed fruits, Yogurt", 
                 preparation="Blend fruits with yogurt", 
                 suitable_for=["Celiac Disease", "IBS"]),
            Meal(name="Vegetable Stir Fry", protein=10, carbs=30, fat=10, 
                 ingredients="Mixed vegetables, Soy sauce", 
                 preparation="Stir-fry vegetables with soy sauce", 
                 suitable_for=["Diabetes", "Hypertension", "IBS"])
        ]
        for meal in meals:
            self.declare(meal)

    @Rule(Fact(cal=MATCH.cal), 
          Fact(protein=MATCH.protein), 
          Fact(carbs=MATCH.carbs), 
          Fact(fat=MATCH.fat), 
          UserProfile(allergic=MATCH.allergic, diseases=MATCH.diseases))
    def generate_meal_plan(self, cal, protein, carbs, fat, allergic, diseases):
        meal_plan = []
        total_protein, total_carbs, total_fat = 0, 0, 0
        
        # Filter meals based on allergies and diseases
        meals = [fact for fact in self.facts.values() if isinstance(fact, Meal)]
        if allergic:
            meals = [meal for meal in meals if allergic not in meal.ingredients]
        
        if diseases:
            meals = [meal for meal in meals if all(disease in meal.suitable_for for disease in diseases)]

        # Generate a meal plan to match macros
        for meal in meals:
            if total_protein < protein and total_carbs < carbs and total_fat < fat:
                meal_plan.append(meal)
                total_protein += meal.protein
                total_carbs += meal.carbs
                total_fat += meal.fat
        
        self.meal_plan = "\n".join([f"{meal.name}: {meal.ingredients}\nPreparation: {meal.preparation}" for meal in meal_plan])

class NutritionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Nutrition CalculatorV1.0")
        self.engine = NutritionExpert()

        self.create_widgets()

    def create_widgets(self):
        self.frame1 = ttk.Frame(self.root)
        self.frame2 = ttk.Frame(self.root)
        self.frame3 = ttk.Frame(self.root)
        self.frame4 = ttk.Frame(self.root)
        
        self.frame1.grid(row=0, column=0, sticky='nsew')
        self.frame2.grid(row=0, column=0, sticky='nsew')
        self.frame3.grid(row=0, column=0, sticky='nsew')
        self.frame4.grid(row=0, column=0, sticky='nsew')

        self.create_input_screen()
        self.create_result_screen()
        self.create_allergy_screen()
        self.create_meal_screen()
        
        self.show_frame(self.frame1)

    def create_input_screen(self):
        self.gender_var = tk.StringVar()
        self.weight_var = tk.DoubleVar(value=60)  # Default weight
        self.height_var = tk.DoubleVar(value=170)  # Default height
        self.age_var = tk.IntVar(value=30)  # Default age
        self.activity_var = tk.StringVar()
        self.goal_var = tk.StringVar()
        self.pregnancy_var = tk.StringVar(value='no')
        
        ttk.Label(self.frame1, text="Gender:").grid(column=0, row=0, padx=5, pady=5)
        ttk.Radiobutton(self.frame1, text="Male", variable=self.gender_var, value="male").grid(column=1, row=0, padx=5, pady=5)
        ttk.Radiobutton(self.frame1, text="Female", variable=self.gender_var, value="female").grid(column=2, row=0, padx=5, pady=5)
        ttk.Label(self.frame1, text="Weight (kg):").grid(column=0, row=1, padx=5, pady=5)
        ttk.Entry(self.frame1, textvariable=self.weight_var, width=10).grid(column=1, row=1, padx=5, pady=5)

        ttk.Label(self.frame1, text="Height (cm):").grid(column=0, row=2, padx=5, pady=5)
        ttk.Entry(self.frame1, textvariable=self.height_var, width=10).grid(column=1, row=2, padx=5, pady=5)

        
        ttk.Label(self.frame1, text="Age:").grid(column=0, row=3, padx=5, pady=5)
        ttk.Spinbox(self.frame1, from_=18, to=100, textvariable=self.age_var, width=5).grid(column=1, row=3, padx=5, pady=5)
        
        ttk.Label(self.frame1, text="Activity Level:").grid(column=0, row=4, padx=5, pady=5)
        ttk.Combobox(self.frame1, textvariable=self.activity_var, values=["none", "light", "moderate", "very_active", "extremely_active"]).grid(column=1, row=4, padx=5, pady=5)
        
        ttk.Label(self.frame1, text="Goal:").grid(column=0, row=5, padx=5, pady=5)
        ttk.Combobox(self.frame1, textvariable=self.goal_var, values=["cutting", "bulking", "maintenance"]).grid(column=1, row=5, padx=5, pady=5)

        ttk.Label(self.frame1, text="Are you pregnant (if female)?").grid(column=0, row=6, padx=5, pady=5)
        ttk.Radiobutton(self.frame1, text="Yes", variable=self.pregnancy_var, value="yes").grid(column=1, row=6, padx=5, pady=5)
        ttk.Radiobutton(self.frame1, text="No", variable=self.pregnancy_var, value="no").grid(column=2, row=6, padx=5, pady=5)
        
        ttk.Button(self.frame1, text="Calculate", command=self.calculate).grid(column=0, row=7, columnspan=3, padx=5, pady=5)

    def create_result_screen(self):
        self.result_var = tk.StringVar()
        ttk.Label(self.frame2, textvariable=self.result_var).grid(column=0, row=0, padx=5, pady=5)
        
        ttk.Button(self.frame2, text="Generate Meals", command=self.ask_allergy_disease).grid(column=0, row=1, padx=5, pady=5)

    def create_allergy_screen(self):
        self.allergic_var = tk.StringVar()
        self.diseases_var = []

        ttk.Label(self.frame3, text="Do you have any allergies?").grid(column=0, row=0, padx=5, pady=5)
        ttk.Entry(self.frame3, textvariable=self.allergic_var).grid(column=1, row=0, padx=5, pady=5)
        
        ttk.Label(self.frame3, text="Select any diseases that affect your meals:").grid(column=0, row=1, padx=5, pady=5)

        diseases = ["Diabetes", "Hypertension", "Celiac Disease", "Irritable Bowel Syndrome (IBS)", "Heart Disease"]
        for i, disease in enumerate(diseases):
            var = tk.IntVar()
            ttk.Checkbutton(self.frame3, text=disease, variable=var).grid(column=1, row=i+1, padx=5, pady=2, sticky='w')
            self.diseases_var.append(var)

        ttk.Button(self.frame3, text="Generate Meal Plan", command=self.generate_meal_plan).grid(column=0, row=len(diseases)+2, columnspan=2, padx=5, pady=5)

    def create_meal_screen(self):
        self.meal_var = tk.StringVar()
        ttk.Label(self.frame4, textvariable=self.meal_var).grid(column=0, row=0, padx=5, pady=5)

    def show_frame(self, frame):
        frame.tkraise()

    def calculate(self):
        self.engine.reset()
        self.engine.declare(UserProfile(gender=self.gender_var.get(),
                                        weight=self.weight_var.get(),
                                        height=self.height_var.get(),
                                        age=self.age_var.get(),
                                        activity_level=self.activity_var.get(),
                                        goal=self.goal_var.get(),
                                        pregnancy=self.pregnancy_var.get()))
        self.engine.run()
        self.result_var.set(f"Calories: {self.engine.cal:.2f}\nProtein: {self.engine.protein:.2f} g\nCarbs: {self.engine.carbs:.2f} g\nFat: {self.engine.fat:.2f} g\nBMI: {self.engine.bmi:.2f}")
        self.show_frame(self.frame2)

    def ask_allergy_disease(self):
        self.show_frame(self.frame3)

    def generate_meal_plan(self):
        self.engine.reset()
        allergic = self.allergic_var.get()
        diseases = [disease for var, disease in zip(self.diseases_var, ["Diabetes", "Hypertension", "Celiac Disease", "Irritable Bowel Syndrome (IBS)", "Heart Disease"]) if var.get()]
        self.engine.declare(UserProfile(allergic=allergic, diseases=diseases))
        self.engine.run()
        self.meal_var.set(self.engine.meal_plan)
        self.show_frame(self.frame4)

if __name__ == "__main__":
    root = tk.Tk()
    app = NutritionApp(root)
    root.mainloop()
